In [2]:
import requests
import json
import os
import logging
import queue
import threading
import html2md
import concurrent.futures
import time
import re 

from threading import Lock
from random import choice
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from boilerpy3 import extractors
from collections import defaultdict


In [3]:
q = queue.Queue()

valid_proxies = []

with open ("proxy_list.txt","r") as f:
    proxies = f.read().split("\n")
    for p in proxies:
        # print(p)
        q.put(p)
lock = threading.Lock()

In [4]:
def check_proxies():
    global q
    while not q.empty():
        lock.acquire()
        proxy = q.get()
        lock.release()
        try:
            res = requests.get("http://ipinfo.io/json",
                               proxies={
                                   "http": proxy,
                                   "https": proxy
                               },
                               timeout=5)
        except:
            continue
        if res.status_code == 200:
            print(proxy)
            valid_proxies.append(proxy)
    with open("valid_proxies.txt","w+") as f:
        f.write("\n".join(valid_proxies))

In [5]:
for _ in range(100):
    threading.Thread(target=check_proxies).start()

35.79.120.242:3128
54.248.238.110:80
52.196.1.182:80
35.72.118.126:80
3.37.125.76:3128
46.51.249.135:3128
12.186.205.122:80
204.236.176.61:80
103.163.51.254:80
202.131.65.110:80
3.123.150.192:3128
13.37.59.99:3128
15.236.106.236:3128
13.37.89.201:3128
128.199.202.122:8080
103.127.1.130:80
3.122.84.99:3128
13.38.176.104:3128
3.127.62.252:80
38.145.211.246:8899
161.35.70.249:3128
185.247.18.200:8888
157.245.14.43:8888
18.117.123.146:8080
13.59.156.167:3128
3.21.101.158:3128
44.195.247.145:80
52.67.10.183:80
18.228.198.164:80
133.18.234.13:80
84.39.112.144:3128
138.201.182.222:80
39.109.113.97:3128
175.139.233.76:80
51.89.14.70:80
83.98.243.181:80
178.128.113.118:23128
47.74.152.29:8888
123.30.154.171:7777
154.38.171.242:80
35.185.196.38:3128
195.114.209.50:80
18.185.169.150:3128
114.156.77.107:8080
74.48.7.43:80
47.88.3.19:8080
38.145.211.247:8899
117.250.3.58:8080
139.59.1.14:8080
3.212.148.199:3128
18.228.149.161:80
54.233.119.172:3128
47.56.110.204:8989
152.32.243.60:8081
190.103.177.

In [6]:
avail_proxies = []
with open("valid_proxies.txt", "r+") as f:
    avail_proxies = f.read().split("\n")
    # avail_proxies.remove("")
    
len(avail_proxies)

63

In [43]:

# book_link_pattern = re.compile(r'/core/books/[^"]+')
# # Store unique book links
# book_links = set()

# def extract_book_links(html):
#     links = book_link_pattern.findall(html)
#     for link in links:
#         book_links.add(link)

# def extract_urls(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, "html.parser")
#     links = soup.find_all("a", class_="part-link", href=re.compile("^/core/books/"))
#     urls = [link["href"] for link in links]
#     return urls

# def crawl_open_access():
#     url = f"https://www.cambridge.org/core/publications/open-access/listing?aggs%5BproductTypes%5D%5Bfilters%5D=BOOK&aggs%5BproductSubject%5D%5Bfilters%5D=66FF02B2A4F83D9A645001545197F287&searchWithinIds=7275BA1E84CA769210167A6A66523B47"
#     all_urls = []

#     try:
#         urls = extract_urls(url)
#     except Exception as e:
#         print(f"Failed to crawl page | Error: {e}")
#         return all_urls

#     print(f"Page 1 - {len(urls)} links")
#     all_urls.extend(urls)

#     with open("cambridge_urls_medicine.txt", "a") as f:
#         f.write("\n".join(urls))

#     return all_urls

In [65]:
book_link_pattern = re.compile(r'/core/books/[^"]+')

# Store unique book links
book_links = set()

def rotate_proxy():
    if valid_proxies:
        proxy = choice(avail_proxies)
        return {"http": proxy, "https": proxy}
    else:
        return None

def extract_book_links(html):
    links = book_link_pattern.findall(html)
    for link in links:
        book_links.add(link)

def extract_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", class_="part-link", href=re.compile("^/core/books/"))
    urls = [link["href"] for link in links]
    return urls

def crawl_open_access():
    page_num = 1

    all_urls = []
    while True:
        url = f"https://www.cambridge.org/core/publications/open-access/listing?aggs%5BproductTypes%5D%5Bfilters%5D=BOOK&aggs%5BproductSubject%5D%5Bfilters%5D=7C9FB6788DD8D7E6696263BC774F4D5B&pageNum={page_num}&searchWithinIds=7275BA1E84CA769210167A6A66523B47"
        try:
            urls = extract_urls(url)
        except Exception as e:
            print(f"Failed to crawl page {page_num} | Error {e}")
            break
        print(f"Page {page_num} - {len(urls)} links")
        all_urls.extend(urls)
        page_num += 1
        with open("cambridge_law.txt", "a") as f:
            f.write("\n".join(urls))
        if len(urls) == 0:
            print(f"Stop at page {page_num} | Find no more link")
            break
    return all_urls


In [66]:
allu = crawl_open_access()

Page 1 - 20 links
Page 2 - 20 links
Page 3 - 20 links
Page 4 - 20 links
Page 5 - 20 links
Page 6 - 20 links
Page 7 - 3 links
Page 8 - 0 links
Stop at page 9 | Find no more link


In [67]:
def extract_book_urls(url):
    try:
        response = requests.get(url, timeout=120)
        soup = BeautifulSoup(response.content, "html.parser")
        
        coming_soon_span = soup.find('span', class_='productStatus ')
        if coming_soon_span is not None:
            return ["""This book [{" ".join(url.split("/")[3:])}] coming soon book"""]

        links = soup.find_all("a", class_="part-link", href=re.compile("^/core/books/"))
        urls = {}

        for link in links:
            if link.parent.name == "h5":
                parent = "root" 
            elif link.parent.name == "h6":
                parent = urls["root"][-1]
            urls.setdefault(parent, []).append(link["href"])
        return urls
    except requests.exceptions.Timeout:
        logging.error(f"Book {url} timed out")
        return []
    except requests.exceptions.ConnectionError:
        logging.error(f"Book {url} connection error")
        return []
    except Exception as e:
        logging.error(f"Book {url} failed | Error {e}")
        return []
        
def add_to_json(file_path, key, value):
    if os.path.isfile(file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
    else:
        with open(file_path, 'w') as f:
            data = {}
            
    data[key] = value
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def main():
    with open("result.json", 'w') as fp:
        fp.write("{}")

    with open("cambridge_law.txt", "r") as f:
        oalinks = f.read().split("\n")
        book_tree = {}
        domain = "https://www.cambridge.org"
        
        for link in tqdm(oalinks, unit="book", total=len(oalinks), desc="Processing data"):
            book_url = f"{domain}{link}" 
            book_tree[link] = extract_book_urls(book_url)                
            tqdm.write(f"Processing link: {book_url}")
            if len(book_tree[link]) == 0:
                tqdm.write(f"""Book {" ".join(link.split("/")[3:])} coming soon""")
            else: tqdm.write(f"""\tDiscovered {len(book_tree[link]["root"])}  links""")
            time.sleep(1) 
            add_to_json(file_path="result.json", key=link, value=book_tree[link])


In [68]:
main()

Processing data:   0%|          | 0/117 [00:00<?, ?book/s]

Processing link: https://www.cambridge.org/core/books/digital-sovereignty-in-the-brics-countries/27E9FD7E4579C76C8D4BA52F7670B431
Book digital-sovereignty-in-the-brics-countries 27E9FD7E4579C76C8D4BA52F7670B431 coming soon
Processing link: https://www.cambridge.org/core/books/customary-international-law-and-its-interpretation-by-international-courts/591B0766E4F1F26AB38D585D97BF54BD
Book customary-international-law-and-its-interpretation-by-international-courts 591B0766E4F1F26AB38D585D97BF54BD coming soon
Processing link: https://www.cambridge.org/core/books/homelessness-liberty-and-property/2A5A5FE412B757D018B16530EA2C0A0C
Book homelessness-liberty-and-property 2A5A5FE412B757D018B16530EA2C0A0C coming soon
Processing link: https://www.cambridge.org/core/books/labour-exploitation-modern-slavery-and-unfree-labour/EA59DEE009388D941F71E2BBE14B83B8
Book labour-exploitation-modern-slavery-and-unfree-labour EA59DEE009388D941F71E2BBE14B83B8 coming soon
Processing link: https://www.cambridge.org

In [69]:

with open('result.json', 'r') as f:
    data = json.load(f)
length = len(data)
print(length)


117


In [70]:
with open('result.json', 'r') as f:
    data = json.load(f)
    # Remove URLs with empty list or array
    data = {key: value for key, value in data.items() if value}

    for key, value in data.items():
        print(key)
        for item in value:
            print(item)
# Save the updated data into a new JSON file
with open('updated_result.json', 'w') as f:
    json.dump(data, f, indent=4)

/core/books/cambridge-handbook-of-facial-recognition-in-the-modern-state/C4C7A2642407D6E58F9943E1F6F0A745
root
/core/books/cambridge-handbook-of-facial-recognition-in-the-modern-state/facial-recognition-technology-in-context/A4F5E2C52EF9CFD27E8F04D0DD60074D
/core/books/cambridge-handbook-of-facial-recognition-in-the-modern-state/facial-recognition-technology-across-the-globe/4EDBBC2AAA12A3640A5DDF962462E958
/core/books/beyond-expropriation-without-compensation/00D7E03A4D13812F1A793C844C62F638
root
/core/books/beyond-expropriation-without-compensation/introduction/0A4EC32459FF3C6FA7570063E0A7DFAF
/core/books/beyond-expropriation-without-compensation/rights-and-wrongs-of-south-african-property-law/F390E714243F8F34367E6AE9D6B8D01C
/core/books/beyond-expropriation-without-compensation/potentials-and-pitfalls-of-south-african-land-reform/21FF0413AD22FE90ABE5318E7EEF7BC2
/core/books/beyond-expropriation-without-compensation/imagining-alternative-futures-of-redistributive-justice-in-south-afr

In [71]:
with open('updated_result.json', 'r') as f:
    data = json.load(f)
length = len(data)

print(length)

90


In [72]:

# Set to store processed URLs
processed_urls = set()

# Lock for thread-safe access to the set
lock = Lock()
def extract_and_save_content(domain, data_file, output_folder, num_threads):
    with open(data_file, 'r') as f:
        data = json.load(f)

    extractor = extractors.ArticleExtractor()

    # Create a thread pool with the specified number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit tasks to the thread pool
        futures = []
        for root_url, value in tqdm(data.items(), desc="Extracting and saving content"):
            full_url = domain + root_url
            if full_url not in processed_urls:  # Check if URL is already processed
                # Submit the task to the thread pool
                future = executor.submit(process_url, full_url, extractor, output_folder, root_url)
                futures.append(future)

            children_urls = value.get('root', [])

            for child_url in children_urls:
                full_url = domain + child_url
                if full_url not in processed_urls:  # Check if URL is already processed
                    # Submit the task to the thread pool
                    future = executor.submit(process_url, full_url, extractor, output_folder, root_url)
                    futures.append(future)

                    grandchild_urls = value.get(child_url, [])

                    for grandchild_url in grandchild_urls:
                        full_url = domain + grandchild_url
                        if full_url not in processed_urls:  # Check if URL is already processed
                            # Submit the task to the thread pool
                            future = executor.submit(process_url, full_url, extractor, output_folder, root_url)
                            futures.append(future)

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

    # Sort the processed URLs
    sorted_urls = sorted(processed_urls)

    # Print the sorted URLs
    print("Sorted URLs:")
    for url in sorted_urls:
        print(url)
def process_url(full_url, extractor, output_folder, root_url):
    try:
        response = requests.get(full_url)
        response.raise_for_status()
        content = extractor.get_content(response.text)
        book_title = root_url.split("/")[3]  # Extract book title from root_url
        chapter_title = full_url.split("/")[-2]  # Extract chapter title from full_url
        save_content_with_figure_urls(content, output_folder, book_title, chapter_title, response.text)
        with lock:
            processed_urls.add(full_url)
    except requests.exceptions.RequestException as e:
        logging.error(f"Error occurred while processing {full_url}: {e}")
    except Exception as e:
        logging.error(f"Error occurred while processing {full_url}: {e}")

def save_content_with_figure_urls(content, output_folder, name, chapter_title, response_text):
    figure_urls = extract_figure_urls(response_text)
    if content is None:
        content = ""
    content_with_figure_urls = content + "\n\nFigure URLs:\n" + "\n".join(figure_urls) if figure_urls else content
    markdown_content = html2md.convert(content_with_figure_urls)
    filename = os.path.join(output_folder, name, f"{chapter_title}.md")
    os.makedirs(os.path.dirname(filename), exist_ok=True)  # Create book folder if needed
    with open(filename, "a") as f:
        f.write(markdown_content)
        f.write("\n\n")

def extract_figure_urls(response_text):
    soup = BeautifulSoup(response_text, "html.parser")
    figures = soup.find_all("div", class_="fig")
    figure_urls = []
    for figure in figures:
        img = figure.find("img")
        if img and "data-src" in img.attrs:
            caption = figure.find("span", class_="caption")
            if caption:
                figure_urls.append(f"{caption.get_text()} - {img['data-src']}")
            else:
                span = figure.find("span")
                if span:
                    figure_urls.append(f"{span.get_text()} - {img['data-src']}")
                else:
                    figure_urls.append(f"{img['data-src']}")
    return figure_urls

# Configure logging
logging.basicConfig(filename='error_md.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

In [73]:
# Call the function with the desired number of threads
extract_and_save_content(
    domain="https://www.cambridge.org",
    data_file="updated_result.json",
    output_folder="cambridge-law-books-md",
    num_threads= 200  # Specify the number of threads you want to use
)

Extracting and saving content:   0%|          | 0/90 [00:00<?, ?it/s]

Sorted URLs:
https://www.cambridge.org/core/books/5g-and-beyond/5g-and-beyond/15A4769714C13D6E7469930788E6DCCA
https://www.cambridge.org/core/books/5g-and-beyond/5g-and-beyond/A90B7B8BD8A7F6AF7FFDDFAFC4A18ED5
https://www.cambridge.org/core/books/5g-and-beyond/AFF9EE741CD0CF1B28E8B698F985E0C1
https://www.cambridge.org/core/books/5g-and-beyond/acknowledgments/D21CF3358235847F682DA19F7CAE7F5A
https://www.cambridge.org/core/books/5g-and-beyond/antitrust-convergence-on-substantive-norms-for-sep-licensing-negotiations/A97F752332F8BD95A9E98D87E5C9F070
https://www.cambridge.org/core/books/5g-and-beyond/cellular-sep-royalties-and-5g/8755988D408D15C5BD5F64C7DAFA9696
https://www.cambridge.org/core/books/5g-and-beyond/chinas-practice-of-antisuit-injunctions-in-standardessential-patent-litigation/22512315C001CFC69054198ED2057AF3
https://www.cambridge.org/core/books/5g-and-beyond/contents/CA7423278814545F6AF266793538641F
https://www.cambridge.org/core/books/5g-and-beyond/contributors/23B8A5FB02B3C0C

In [55]:
!rm -rf updated_result.json
!rm -rf result.json